In [ ]:
import anndata as ad
import pandas as pd
import numpy as np
import scanpy as sc
import decoupler as dc
import os

In [ ]:
def validate_input_arguments (arguments_list):
    if arguments_list["out_path"] is None:
        print("Please provide an output path")
    elif arguments_list["out_path"][-1] != "/":
        arguments_list["out_path"] = arguments_list["out_path"] + "/"

    if arguments_list["celltype"] is None:
        print("Please provide the name of the metadata field containing cell type annotations")

    if arguments_list["condition"] is None:
        print("Please provide the name of the metadata field containing condition annotations")

    if arguments_list["organism"] is None:
        arguments_list["organism"] = "human"

    if arguments_list["comparison_list"] is None:
        arguments_list["comparison_list"] = np.nan

    if arguments_list["logfc"] is None:
        arguments_list["logfc"] = 0.0

    if arguments_list ["pval"] is None:
        arguments_list["pval"] = 0.05

    if arguments_list["reg"] is None:
        arguments_list["reg"] = load_dorothea_regulon(arguments_list["organism"])

    elif isinstance(arguments_list["reg"], str):
        arguments_list["reg"] = pd.read_csv(arguments_list["reg"], index_col=0)
        arguments_list["reg"] = pd.DataFrame.rename(arguments_list["reg"], columns={"source" : "tf"})

    if not "tf" in arguments_list["reg"] and "target" in arguments_list["reg"] and "weight"in arguments_list["reg"]:
        raise Exception("Not all necessary columns found in regulon table! Please make sure that the regulon has the columns source, target and weight!")
    
 
   
    return(arguments_list)



In [ ]:
def AverageExpression(anndataobject, celltype = None, outpath = None):
    gene_ids = anndataobject.var.index.values
    #cluster line even necessary if not returned?
    #clusters = anndataobject.obs[celltype].cat.categories
    obs = anndataobject[:,gene_ids].X.toarray()
    obs = np.expm1(obs)
    sub_object = pd.DataFrame(obs,columns=gene_ids,index= anndataobject.obs[celltype])
    sub_object = sub_object.groupby(level=0, observed=False).mean()
    sub_object.T.to_csv(outpath + "average_gene_expression_by_cluster_exp.csv")

    return sub_object.T


In [ ]:
#not per cluster but cluster and pval etc need to be added to csv (check against specific marker csv from lr2tf test run in R)

def get_significant_tfs_single(anndataobject, celltype, condition, out_path, pval, logfc):
    
    #does not work if condition is None
    single_result_path = out_path + condition 
    if not os.path.isdir(single_result_path):
        os.mkdir(single_result_path)
    
    #sc.pp.scale(tf_activities)
    #or sc.pp.normalize_total(anndataobject)
    # sc.pp.log1p(anndata_object) 
    

    number_of_clusters = len(anndataobject.obs["new_annotation"].cat.categories) 
    
    
    sc.tl.rank_genes_groups(tf_activities, groupby= "new_annotation", reference="rest", method="wilcoxon", key_added="wilcoxon_markers", corr_method= "bonferroni")
    #sc.tl.filter_rank_genes_groups(anndataobject, min_in_group_fraction=0, key="wilcoxon_markers", key_added= "wilcoxon_markers_filtered")
     
    sc.tl.rank_genes_groups(tf_activities, groupby= "new_annotation", reference="rest", method="t-test_overestim_var", key_added="t_test_overestim_var_markers")
    #sc.tl.filter_rank_genes_groups(anndataobject, min_in_group_fraction=0, key="t_test_overestim_var_markers", key_added="t_test_overestim_filtered")

    #FindAllMarkers(seuratobject, only.pos = TRUE, min.pct = 0, logfc.threshold = 0, verbose = FALSE)

    result1 = tf_activities.uns['wilcoxon_markers']
    groups = result1['names'].dtype.names
    anndataobject_markers = pd.DataFrame(
    {group + '_' + key[:1]: result1[key][group]
    for group in groups for key in ['names','logfoldchanges','pvals','pvals_adj']})

    result2 = tf_activities.uns['t_test_overestim_var_markers']
    groups = result2['names'].dtype.names
    anndataobject_markers = pd.DataFrame(
    {group + '_' + key[:1]: result2[key][group]
    for group in groups for key in ['names','logfoldchanges','pvals','pvals_adj']})

    anndataobject_markers_wilcoxon = sc.get.rank_genes_groups_df(tf_activities, group = None, log2fc_min=0, key="wilcoxon_markers")
    anndataobject_markers_wilcoxon.to_csv("specific_markers_wilcoxon_test.csv",index=0)

    anndataobject_markers_t_over = sc.get.rank_genes_groups_df(tf_activities, group = None, log2fc_min=0, key="t_test_overestim_var_markers")
    anndataobject_markers_t_over.to_csv("specific_markers_t_test_overestim_test.csv",index=0)

    

    res = list()

    return res

In [130]:
#cell is only for trying out stuff
#anndataobject = ad.read_h5ad("LR2TF_test_run/anndata_object.h5ad")

#tf_activities = ad.read_csv("decoupler_results.csv")
#tf_activities.obsm = anndataobject.obsm
#tf_activities.uns = anndataobject.uns
#tf_activities.obs = anndataobject.obs

#sc.pp.scale(tf_activities,zero_center=True)
#del tf_activities.var["mean"]
#del tf_activities.var["std"]
#or sc.pp.normalize_total(tf_activities)

#expects logarithmized data
#sc.tl.rank_genes_groups(tf_activities, groupby= "new_annotation", reference="rest", method="wilcoxon", key_added="wilcoxon_markers", corr_method= "bonferroni")
#sc.tl.filter_rank_genes_groups(anndataobject, min_in_group_fraction=0, key="wilcoxon_markers", key_added= "wilcoxon_markers_filtered")
     
#sc.tl.rank_genes_groups(tf_activities, groupby= "new_annotation", reference="rest", method="t-test_overestim_var", key_added="t_test_overestim_var_markers")
#sc.tl.filter_rank_genes_groups(anndataobject, min_in_group_fraction=0, key="t_test_overestim_var_markers", key_added="t_test_overestim_filtered")

#FindAllMarkers(seuratobject, only.pos = TRUE, min.pct = 0, logfc.threshold = 0, verbose = FALSE)

#result1 = tf_activities.uns['wilcoxon_markers']
#groups = result1['names'].dtype.names
#anndataobject_markers = pd.DataFrame(
#{group + '_' + key[:1]: result1[key][group]
#for group in groups for key in ['names','logfoldchanges','pvals','pvals_adj']})

#result2 = tf_activities.uns['t_test_overestim_var_markers']
#groups = result2['names'].dtype.names
#anndataobject_markers = pd.DataFrame(
#{group + '_' + key[:1]: result2[key][group]
#for group in groups for key in ['names','logfoldchanges','pvals','pvals_adj']})

#anndataobject_markers_wilcoxon = sc.get.rank_genes_groups_df(tf_activities, group = None, log2fc_min=0, key="wilcoxon_markers")
#anndataobject_markers_wilcoxon.to_csv("specific_markers_wilcoxon_test.csv",index=0)

#anndataobject_markers_t_over = sc.get.rank_genes_groups_df(tf_activities, group = None, log2fc_min=0, key="t_test_overestim_var_markers")
#anndataobject_markers_t_over.to_csv("specific_markers_t_test_overestim_test.csv",index=0)
    




5

In [ ]:
anndataobject = ad.read_h5ad("LR2TF_test_run/anndata_object.h5ad")

tf_activities = ad.read_csv("decoupler_results.csv")
tf_activities.obsm = anndataobject.obsm
tf_activities.uns = anndataobject.uns
tf_activities.obs = anndataobject.obs
tf_activities.X
sc.pp.scale(tf_activities)
tf_activities.X


In [ ]:
#test if subsetting object and using sc.pp.calculate_qc_metrics yields same results (yes it does)

#anndataobject = ad.read_h5ad("LR2TF_test_run/anndata_object.h5ad")
#anndataobject.obs["new_annotation"]
#neural = anndataobject[anndataobject.obs.new_annotation == "Neural"]
#megakaryocyte = anndataobject[anndataobject.obs.new_annotation == "Neural"]
#msc = anndataobject[anndataobject.obs.new_annotation == "Neural"]
#fibroblast = anndataobject[anndataobject.obs.new_annotation == "Neural"]
#myeloid = anndataobject[anndataobject.obs.new_annotation == "Neural"]

#test_neural = sc.pp.calculate_qc_metrics(neural, inplace=True)
#neural.var.to_csv("test_neural")

In [ ]:
#ignore extra tfs from decoupler while writing script 

def tf_activity_analysis (anndataobject, tf_activities = "decoupler_results.csv", arguments_list = None):
    
    if (isinstance(anndataobject, str)):
        anndataobject = ad.read_h5ad(anndataobject)

    arguments_list = validate_input_arguments(arguments_list)

    if not os.path.isdir(arguments_list["out_path"]):
        os.mkdir(arguments_list["out_path"])
        tf_path = arguments_list["out_path"] + "TF_results/"
        os.mkdir(tf_path)
    else:
        tf_path = arguments_list["out_path"] + "TF_results/"

    #add for tf_activities = 0: access the tf data added by running decoupler
    if isinstance(tf_activities, str):
        tf_activities = ad.read_csv(tf_activities)
        tf_activities.obsm = anndataobject.obsm
        tf_activities.uns = anndataobject.uns
        tf_activities.obs = anndataobject.obs


    anndataobject.obs["condition"] = arguments_list["condition"] 
    anndataobject.obs["cell_type"] = arguments_list["celltype"]
    anndataobject.obs["comparison_list"] = arguments_list["comparison_list"]

    if not np.isnan(arguments_list["comparison_list"]):
        if len(arguments_list["comparison_list"]) > 0 & len(anndataobject.obs["comparison_list"]) < 2:
            arguments_list["comparison_list"] <- np.nan
            print("Only one condition was found in the data, although a list of comparisons was provided. The analyses are performed only for the present condition!")

    #code for single condition  analysis

    if np.isnan(arguments_list["comparison_list"]):
        result_list = []
        gene_expression_list = []
        CTR_cluster_list = []
        intranet_cluster_list = []

        #Idents(object = seuratobject) <- arguments_list$condition
        #seuratobject_list <- SplitObject(seuratobject, split.by = arguments_list$condition)
        #for (name in names(seuratobject_list)) {
        #sub_object <- seuratobject_list[[name]]
        anndataobject_list = list[anndataobject.obs.condition]

        sub_object = anndataobject
        sub_object.uns["Average_Expression"] = AverageExpression(sub_object, celltype = arguments_list["celltype"], outpath= arguments_list["out_path"])

        #add name into parameters later 
        tf_activity_scores = get_significant_tfs_single(tf_activities, arguments_list["celltype"], arguments_list["condition"], tf_path, pval = arguments_list["pval"], logfc = arguments_list["logfc"])
        #result_list["name"] = tf_activity_scores
        #gene_expression_list[name + "_average_expression"] = sub_object.layers["cell_type"]
        #if (arguments_list["organism"] == "human"):
        #CTR_cluster_list["name"] = generate_CrossTalkeR_input(tf_activity_scores[

    return(sub_object)

In [ ]:
sub_object = tf_activity_analysis(anndataobject= "LR2TF_test_run/anndata_object.h5ad", arguments_list= {"out_path" : "script_test", "celltype" : "new_annotation", "condition" : "control", "organism" : None, "comparison_list" : None, "logfc" : None, "pval" : None, "reg" : "filterd_regulon.csv"})

In [ ]:
#anndataobject[:,"ISG15"].X.todense().sum()
#sub_object.uns["Average_Expression"]